In [ ]:
#1


In [ ]:
#2# 9  FastAPI Trading Data Backend for Next.js Integration
# fake data tho?
from fastapi import FastAPI, HTTPException, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from pydantic import BaseModel
from typing import Dict, List, Optional, Any
import uvicorn
import asyncio
import nest_asyncio
import json
from datetime import datetime, timedelta
import threading
import time
import pandas as pd
import numpy as np


app = FastAPI(
    title="QUBT Trading API",
    description="Real-time technical indicators and options spread suggestions",
    version="1.0.0"
)

# Configure CORS for Next.js
app.add_middleware(
    CORSMiddleware,
    allow_origins=[
        "http://localhost:3000",  # Next.js dev server
        "http://localhost:3001",  # Alternative Next.js port
        "https://your-nextjs-app.vercel.app",  # Production domain
        "https://*.vercel.app",  # All Vercel deployments
    ],
    allow_credentials=True,
    allow_methods=["GET", "POST", "PUT", "DELETE"],
    allow_headers=["*"],
)

# Global data store (in production, use Redis or database)
trading_data_store = {
    "current_indicators": None,
    "spread_suggestions": None,
    "last_updated": None,
    "is_updating": False
}

# Pydantic models for API responses
class TechnicalIndicators(BaseModel):
    symbol: str
    timestamp: str
    current_price: Dict[str, float]
    indicators: Dict[str, float]

class SpreadSuggestion(BaseModel):
    timeframe: str
    expiration_date: str
    expected_move: float
    call_spread: Dict[str, Any]
    put_spread: Dict[str, Any]
    technical_justification: List[str]

class SpreadAnalysis(BaseModel):
    market_bias: str
    bias_strength: float
    support_levels: List[float]
    resistance_levels: List[float]
    spread_suggestions: List[SpreadSuggestion]
    overall_recommendation: str

class APIResponse(BaseModel):
    success: bool
    data: Optional[Any] = None
    message: Optional[str] = None
    timestamp: str

# Integrated trading functions
def get_current_technical_indicators(symbol: str = "QUBT"):
    """
    Simulate fetching current technical indicators
    In production, replace with actual market data API calls
    """
    # Mock data - replace with actual API calls to your data provider
    current_data = {
        "symbol": symbol,
        "timestamp": datetime.now().isoformat(),
        "current_price": {
            "price": 12.45,
            "change": 0.75,
            "change_percent": 6.4,
            "volume": 1250000,
            "avg_volume": 850000
        },
        "indicators": {
            "EMA_10": 11.85,
            "EMA_20": 11.20,
            "SMA_50": 10.90,
            "RSI": 68.2,
            "MACD": 0.0234,
            "MACD_Signal": 0.0189,
            "MACD_Histogram": 0.0045,
            "BB_Upper": 13.10,
            "BB_Middle": 11.95,
            "BB_Lower": 10.80,
            "Stoch_K": 72.5,
            "Stoch_D": 68.3,
            "Williams_R": -25.4,
            "CCI": 145.6
        }
    }
    return current_data

def suggest_vertical_spreads(current_data):
    """
    Suggest vertical spreads based on technical indicator analysis
    """
    print(f"🎯 VERTICAL SPREAD SUGGESTIONS FOR {current_data['symbol']}")
    
    # Extract key data points
    current_price = current_data.get("current_price", {}).get("price", 0)
    indicators = current_data.get("indicators", {})
    
    # Key technical levels
    ema10 = indicators.get("EMA_10", 0)
    ema20 = indicators.get("EMA_20", 0)
    sma50 = indicators.get("SMA_50", 0)
    rsi = indicators.get("RSI", 50)
    bb_upper = indicators.get("BB_Upper", 0)
    bb_lower = indicators.get("BB_Lower", 0)
    bb_middle = indicators.get("BB_Middle", 0)
    macd = indicators.get("MACD", 0)
    macd_signal = indicators.get("MACD_Signal", 0)
    
    # Determine market bias
    bullish_signals = 0
    bearish_signals = 0
    
    # EMA Analysis
    if ema10 > ema20:
        bullish_signals += 1
    else:
        bearish_signals += 1
    
    # Price vs EMAs
    if current_price > ema10:
        bullish_signals += 1
    else:
        bearish_signals += 1
    
    # RSI Analysis
    if rsi < 30:
        bullish_signals += 1  # Oversold = potential bounce
    elif rsi > 70:
        bearish_signals += 1  # Overbought = potential pullback
    
    # MACD Analysis
    if macd > macd_signal:
        bullish_signals += 1
    else:
        bearish_signals += 1
    
    # Bollinger Band Analysis
    if current_price < bb_lower:
        bullish_signals += 1  # Oversold
    elif current_price > bb_upper:
        bearish_signals += 1  # Overbought
    
    # Determine overall bias
    if bullish_signals > bearish_signals:
        market_bias = "BULLISH"
        bias_strength = bullish_signals / (bullish_signals + bearish_signals)
    else:
        market_bias = "BEARISH"
        bias_strength = bearish_signals / (bullish_signals + bearish_signals)
    
    # Calculate support and resistance levels
    support_levels = []
    resistance_levels = []
    
    # Add EMA levels
    support_levels.extend([ema10, ema20, sma50])
    resistance_levels.extend([ema10, ema20, sma50])
    
    # Add Bollinger Band levels
    support_levels.append(bb_lower)
    resistance_levels.append(bb_upper)
    
    # Filter and sort levels
    support_levels = sorted([level for level in support_levels if level > 0 and level < current_price])
    resistance_levels = sorted([level for level in resistance_levels if level > current_price])
    
    # Calculate expiration dates
    today = datetime.now()
    expirations = {
        "1 Week": today + timedelta(days=7),
        "2 Weeks": today + timedelta(days=14),
        "4 Weeks": today + timedelta(days=28),
        "6 Weeks": today + timedelta(days=42)
    }
    
    # Generate spread suggestions for each timeframe
    spread_suggestions = []
    
    for period, exp_date in expirations.items():
        # Calculate expected price range based on timeframe
        days_to_exp = (exp_date - today).days
        
        # Volatility estimate (simplified)
        if bb_upper > 0 and bb_lower > 0:
            implied_volatility = (bb_upper - bb_lower) / bb_middle
        else:
            implied_volatility = 0.20  # Default 20%
        
        # Expected move calculation
        expected_move = current_price * implied_volatility * np.sqrt(days_to_exp / 365)
        
        # CALL CREDIT SPREADS (Bearish/Neutral)
        if resistance_levels:
            short_call_strike = min(resistance_levels)
        else:
            short_call_strike = current_price + expected_move * 0.5
        
        long_call_strike = short_call_strike + (current_price * 0.05)  # 5% wide
        
        # Adjust strikes to reasonable increments
        short_call_strike = round(short_call_strike * 2) / 2  # Round to nearest $0.50
        long_call_strike = round(long_call_strike * 2) / 2
        
        # PUT CREDIT SPREADS (Bullish/Neutral)
        if support_levels:
            short_put_strike = max(support_levels)
        else:
            short_put_strike = current_price - expected_move * 0.5
        
        long_put_strike = short_put_strike - (current_price * 0.05)  # 5% wide
        
        # Adjust strikes to reasonable increments
        short_put_strike = round(short_put_strike * 2) / 2  # Round to nearest $0.50
        long_put_strike = round(long_put_strike * 2) / 2
        
        # Technical justification
        technical_justification = []
        if short_call_strike > bb_upper:
            technical_justification.append(f"Call spread above Bollinger Upper Band (${bb_upper:.2f})")
        if short_call_strike > ema10:
            technical_justification.append(f"Call spread above EMA 10 (${ema10:.2f})")
        if rsi > 60:
            technical_justification.append(f"RSI suggests potential resistance at {rsi:.1f}")
        if short_put_strike < bb_lower:
            technical_justification.append(f"Put spread below Bollinger Lower Band (${bb_lower:.2f})")
        if short_put_strike > ema20:
            technical_justification.append(f"Put spread above EMA 20 support (${ema20:.2f})")
        
        # Find next Friday for expiration
        days_until_friday = (4 - exp_date.weekday()) % 7
        if days_until_friday == 0 and exp_date.hour >= 16:
            days_until_friday = 7
        friday_date = exp_date + timedelta(days=days_until_friday)
        
        spread_suggestion = {
            "timeframe": period,
            "expiration_date": friday_date.strftime('%Y-%m-%d'),
            "expected_move": round(expected_move, 2),
            "call_spread": {
                "type": "Credit Spread",
                "short_strike": short_call_strike,
                "long_strike": long_call_strike,
                "width": round(long_call_strike - short_call_strike, 2),
                "max_profit": "Premium Collected",
                "max_loss": f"${long_call_strike - short_call_strike:.2f} - Premium",
                "breakeven": f"${short_call_strike:.2f} + Premium"
            },
            "put_spread": {
                "type": "Credit Spread",
                "short_strike": short_put_strike,
                "long_strike": long_put_strike,
                "width": round(short_put_strike - long_put_strike, 2),
                "max_profit": "Premium Collected",
                "max_loss": f"${short_put_strike - long_put_strike:.2f} - Premium",
                "breakeven": f"${short_put_strike:.2f} - Premium"
            },
            "technical_justification": technical_justification
        }
        
        spread_suggestions.append(spread_suggestion)
    
    # Overall recommendation
    if market_bias == "BULLISH":
        overall_recommendation = "Primary Focus: PUT CREDIT SPREADS - Bullish bias suggests selling puts below support"
    else:
        overall_recommendation = "Primary Focus: CALL CREDIT SPREADS - Bearish bias suggests selling calls above resistance"
    
    return {
        "market_bias": market_bias,
        "bias_strength": bias_strength,
        "support_levels": support_levels,
        "resistance_levels": resistance_levels,
        "spread_suggestions": spread_suggestions,
        "overall_recommendation": overall_recommendation,
        "expected_moves": expected_move
    }

# Background task to update data
async def update_trading_data_background(symbol: str):
    """Background task to fetch fresh trading data"""
    try:
        trading_data_store["is_updating"] = True
        
        # Get current technical indicators
        current_data = get_current_technical_indicators(symbol)
        
        # Generate spread analysis
        spread_analysis = suggest_vertical_spreads(current_data)
        
        # Store the data
        trading_data_store["current_indicators"] = current_data
        trading_data_store["spread_suggestions"] = spread_analysis
        trading_data_store["last_updated"] = datetime.now().isoformat()
        
        print(f"✅ Updated trading data for {symbol}")
        
    except Exception as e:
        print(f"❌ Error updating trading data: {e}")
    finally:
        trading_data_store["is_updating"] = False

# API Routes
@app.get("/", response_model=APIResponse)
async def root():
    """Health check endpoint"""
    return APIResponse(
        success=True,
        message="QUBT Trading API is running",
        timestamp=datetime.now().isoformat()
    )

@app.get("/api/indicators/{symbol}", response_model=APIResponse)
async def get_technical_indicators_endpoint(symbol: str, refresh: bool = False):
    """Get current technical indicators for a symbol"""
    try:
        # Force refresh if requested or data is stale
        if refresh or not trading_data_store["current_indicators"] or is_data_stale():
            await update_trading_data_background(symbol.upper())
        
        if not trading_data_store["current_indicators"]:
            raise HTTPException(status_code=404, detail="No indicator data available")
        
        return APIResponse(
            success=True,
            data=trading_data_store["current_indicators"],
            timestamp=datetime.now().isoformat()
        )
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/api/spreads/{symbol}", response_model=APIResponse)
async def get_spread_suggestions_endpoint(symbol: str, refresh: bool = False):
    """Get vertical spread suggestions for a symbol"""
    try:
        # Force refresh if requested or data is stale
        if refresh or not trading_data_store["spread_suggestions"] or is_data_stale():
            await update_trading_data_background(symbol.upper())
        
        if not trading_data_store["spread_suggestions"]:
            raise HTTPException(status_code=404, detail="No spread data available")
        
        return APIResponse(
            success=True,
            data=trading_data_store["spread_suggestions"],
            timestamp=datetime.now().isoformat()
        )
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/api/dashboard/{symbol}", response_model=APIResponse)
async def get_dashboard_data(symbol: str, refresh: bool = False):
    """Get complete dashboard data (indicators + spreads) for Next.js"""
    try:
        # Force refresh if requested or data is stale
        if refresh or not trading_data_store["current_indicators"] or is_data_stale():
            await update_trading_data_background(symbol.upper())
        
        dashboard_data = {
            "symbol": symbol.upper(),
            "indicators": trading_data_store["current_indicators"],
            "spreads": trading_data_store["spread_suggestions"],
            "last_updated": trading_data_store["last_updated"],
            "is_updating": trading_data_store["is_updating"]
        }
        
        return APIResponse(
            success=True,
            data=dashboard_data,
            timestamp=datetime.now().isoformat()
        )
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/api/refresh/{symbol}")
async def refresh_data(symbol: str, background_tasks: BackgroundTasks):
    """Trigger background refresh of trading data"""
    try:
        if trading_data_store["is_updating"]:
            return APIResponse(
                success=False,
                message="Update already in progress",
                timestamp=datetime.now().isoformat()
            )
        
        background_tasks.add_task(update_trading_data_background, symbol.upper())
        
        return APIResponse(
            success=True,
            message=f"Data refresh initiated for {symbol.upper()}",
            timestamp=datetime.now().isoformat()
        )
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/api/status")
async def get_api_status():
    """Get API status and data freshness"""
    return APIResponse(
        success=True,
        data={
            "has_indicator_data": trading_data_store["current_indicators"] is not None,
            "has_spread_data": trading_data_store["spread_suggestions"] is not None,
            "last_updated": trading_data_store["last_updated"],
            "is_updating": trading_data_store["is_updating"],
            "data_age_minutes": get_data_age_minutes()
        },
        timestamp=datetime.now().isoformat()
    )

# WebSocket endpoint for real-time updates
@app.websocket("/ws/{symbol}")
async def websocket_endpoint(websocket, symbol: str):
    """WebSocket for real-time trading data updates"""
    await websocket.accept()
    
    try:
        while True:
            # Send current data
            if trading_data_store["current_indicators"]:
                await websocket.send_json({
                    "type": "indicators",
                    "data": trading_data_store["current_indicators"],
                    "timestamp": datetime.now().isoformat()
                })
            
            if trading_data_store["spread_suggestions"]:
                await websocket.send_json({
                    "type": "spreads", 
                    "data": trading_data_store["spread_suggestions"],
                    "timestamp": datetime.now().isoformat()
                })
            
            # Wait 30 seconds before next update
            await asyncio.sleep(30)
            
    except Exception as e:
        print(f"WebSocket error: {e}")
    finally:
        await websocket.close()

# Utility functions
def is_data_stale(max_age_minutes: int = 15) -> bool:
    """Check if cached data is older than max_age_minutes"""
    if not trading_data_store["last_updated"]:
        return True
    
    last_updated = datetime.fromisoformat(trading_data_store["last_updated"])
    age = datetime.now() - last_updated
    return age.total_seconds() > (max_age_minutes * 60)

def get_data_age_minutes() -> Optional[float]:
    """Get age of cached data in minutes"""
    if not trading_data_store["last_updated"]:
        return None
    
    last_updated = datetime.fromisoformat(trading_data_store["last_updated"])
    age = datetime.now() - last_updated
    return age.total_seconds() / 60

# Startup event
@app.on_event("startup")
async def startup_event():
    """Initialize API on startup"""
    print("🚀 QUBT Trading API starting up...")
    print("📊 Ready to serve Next.js frontend")
    
    # Initialize with QUBT data
    await update_trading_data_background("QUBT")
    nest_asyncio.apply()
# Run the server
if __name__ == "__main__":
    print("🔥 Starting QUBT Trading API Server")
    print("📡 API will be available at: http://localhost:8000")
    print("QUBT data here: http://localhost:8000/api/dashboard/QUBT?refresh=true")
    print("📖 API docs available at: http://localhost:8000/docs")
    print("🔄 WebSocket endpoint: ws://localhost:8000/ws/QUBT")
    print("\n🎯 Integrated Functions:")
    print("   ✅ get_current_technical_indicators()")
    print("   ✅ suggest_vertical_spreads()")
    print("   ✅ Real-time data updates")
    print("   ✅ Background refresh tasks")

    # uvicorn.run(
    #     "main:app",  # Use app instance from current module
    #     host="0.0.0.0",
    #     port=8000,
    #     reload=True,
    #     log_level="info"
    # )



In [ ]:
#3 
# works actually running port
import threading
import nest_asyncio

nest_asyncio.apply()

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")
# 10

import uvicorn



print("🔥 Starting QUBT Trading API Server")
print("📡 API will be available at: http://localhost:8000")
print("📖 API docs available at: http://localhost:8000/docs")
print("🔄 WebSocket endpoint: ws://localhost:8000/ws/QUBT")
# print(f"✅ Updated trading data for {QUBT}")
# print(f"🎯 VERTICAL SPREAD SUGGESTIONS FOR {current_data['QUBT']}")
uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")


# Start server in background
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

print("🔥 Server started! Check http://localhost:8000")


In [ ]:
#4
# post via sql or similar to db